# Transformer 

Transformer 是一种基于 ​​自注意力机制（Self-Attention）​​ 的深度学习模型，由 Google 在 2017 年的论文《Attention Is All You Need》中提出。它彻底改变了自然语言处理（NLP）领域，并逐步扩展到计算机视觉、语音识别等多个领域。

------

## 原理介绍

### 什么是注意力机制

注意力机制（​​Attention Mechanism​​）是深度学习中一种模拟人类视觉或认知聚焦行为的技术，它允许模型在处理数据时动态地​​关注与当前任务最相关的部分​​，而忽略无关信息。其核心思想是​​通过权重分配，强调重要特征，弱化次要特征​​。

#### 核心数学形式​​

注意力机制通常分为三步（以经典的 ​​Scaled Dot-Product Attention​​ 为例）：

+ ​​计算注意力分数​​：
对查询（Query）和键（Key）做相似度计算，得到权重。
$$
Attention Score= \frac {Q⋅K^T} {\sqrt d_k}
$$
（d_k 是 Key 的维度，缩放用于防止梯度消失）
+ 归一化为权重（Softmax）​​：
$$Attention Weights=softmax(Attention Score)$$
+ ​​加权求和值（Value）​​：
$$Output=Attention Weights⋅V$$

#### 举例解释

+ 假设一句话为 "the cat sat on the table", 每一个单词都可以用一个 $1 \times N$ 的嵌入向量表示，这样这句话就表示成了 $X = 6 \times N$ 的矩阵
+ 训练三个权重向量 ($N \times 1$) 分别为 $W_Q, W_K, W_V$, 计算出查询，键和值 $Q = X * W_Q, K = X * W_K, V = X * W_V$
+ 计算每一个单词的注意力得分，并和 V 相乘，得到最终的输出

--------


### 注意力机制的类型

1. 自注意力 (self-attention)
    Query, Key, Value 来自同一输入，用来捕捉序列内部的关系（比如句子中单词的远距离依赖）
    ```python
    self_atten = nn.MultiheadAttention(embed_dim=64, num_head=4)
    output, _ = self_atten(query, key, value)
    ```
2. 交叉注意力 (cross-attention)
    Query 来自当前序列，Key， Value来自预训练的序列 （如机器翻译中源语言到目标语言）
3. 多头注意力 (multi-head attention)
    将 Q/K/V 分成不同的头，并行计算后拼接，实现对不同子空间的特征捕捉能力

--------


In [ ]:
import torch
import torch.nn as nn

import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import dataloader
import numpy as np


# 超参数
batch_size = 64
patch_size = 7      # 将图像分割成 7x7 的patch 小块
embed_size = 64     # 嵌入向量的维度
num_heads = 4       # 注意力头数
num_classes = 10
epochs = 5

# 数据加载
transform = transforms.Compose([
    transforms.ToTensor(),                  # 步骤1: 将图像转为 Tensor 格式
    transforms.Normalize((0.5,), (0.5,))    # 步骤2: 归一化处理，使数据符合均值为0.5，标准差也是0.5
])
